In [1]:
# pip install sodapy

In [2]:
# pip install us

In [3]:
import pandas as pd
from sodapy import Socrata
import requests as rq
import matplotlib.pyplot as plt
import us

In [4]:
# Enrollment data by state
# Source: https://www.kff.org/health-reform/state-indicator/medicaid-expansion-enrollment/?currentTimeframe=0&sortModel=%7B%22colId%22:%22Location%22,%22sort%22:%22asc%22%7D

df_medicaid = pd.read_csv('data/KFF_medicaid_enrollment.csv', header=2).loc[:51] # get rid of junk rows
df_medicaid = df_medicaid.loc[df_medicaid['Location'] != 'United States'] # just states & DC
df_medicaid = df_medicaid.rename(columns={
    'Location':'state',
    'Expanded by September 30th, 2022':'expanded_medicaid'
})
df_medicaid['expanded_medicaid'] = df_medicaid['expanded_medicaid'].replace({'Yes': True, 'No': False})
df_medicaid = df_medicaid[['state', 'expanded_medicaid']]

df_medicaid.head()

,state,expanded_medicaid
1,Alabama,False
2,Alaska,True
3,Arizona,True
4,Arkansas,True
5,California,True


In [5]:
# # Life expectancy by state (2010-2015)
# # Docs: https://dev.socrata.com/foundry/data.cdc.gov/5h56-n989

# client = Socrata("data.cdc.gov", None)
# data = client.get("5h56-n989", where='county_name = "(blank)"')
# df_life_exp_src = pd.DataFrame.from_records(data)

# # df_life_exp = df_life_exp.set_index('state_name')

# df_life_exp = df_life_exp_src.copy()
# df_life_exp = df_life_exp.drop('county_name', axis=1)
# df_life_exp = df_life_exp.rename(columns={'le':'LE'})
# df_life_exp[['LE Range Lower', 'LE Range Upper']] = df_life_exp['le_range'].str.split('-', expand=True)
# df_life_exp = df_life_exp.drop(columns=['le_range', 'se_le'])

# df_life_exp['LE'] = df_life_exp['LE'].astype(float)
# df_life_exp['LE Range Lower'] = df_life_exp['LE Range Lower'].astype(float)
# df_life_exp['LE Range Upper'] = df_life_exp['LE Range Upper'].astype(float)

# df_life_exp.head()


In [6]:
df_life_exp = pd.read_csv('data/cdc_life_exp.csv')
df_life_exp = df_life_exp.rename(columns={'STATE':'state_abbr', 'RATE':'life_exp'})

df_life_exp = df_life_exp.loc[df_life_exp['YEAR'] == max(df_life_exp['YEAR'])]
state_names = []
for s in df_life_exp['state_abbr']:
    state_names.append(us.states.lookup(s).name)
df_life_exp['state'] = state_names
df_life_exp = df_life_exp[['state', 'life_exp']]
df_life_exp.head(5)

,state,life_exp
0,Alabama,73.2
1,Alaska,76.6
2,Arizona,76.3
3,Arkansas,73.8
4,California,79.0


In [18]:
# API endpoint URL for poverty rate data by state
url = 'https://api.census.gov/data/timeseries/poverty/saipe?get=NAME,SAEPOVRTALL_PT&for=state:*&time=2021'

# Send GET request to the API endpoint
response = rq.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Parse the response JSON
    data = response.json()
    columns = data[0]
    df_poverty = pd.read_json(response.json())
else:
    print('Error:', response.status_code)
    
df_poverty


ValueError: Invalid file path or buffer object type: <class 'list'>

In [17]:
type(data)

list

In [ ]:
df_final = df_medicaid.merge(df_life_exp, how='inner', on='state')
df_final = df_final.sort_values('life_exp').reset_index().drop(columns='index')

df_final.head()

In [ ]:
plt.figure(figsize=(10, 5))

for i in range(len(df_final)):
    if df_final.loc[i, 'expanded_medicaid']:
        plt.scatter(i, df_final['life_exp'][i], c='#30b5f2', marker='o')
    else:
        plt.scatter(i, df_final['life_exp'][i], c='#eb4034', marker='D')

plt.xticks(range(len(df_final)), df_final['state'], rotation='vertical')

plt.xlabel('State')
plt.ylabel('Life Expectancy (2020)')
plt.title('Life Expectancy by State')
plt.axvline(x=len(df_final)/2, color='green', linestyle='-')

plt.show()

